## Классификация текстов: спам-фильтр для SMS
 В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

In [34]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import codecs
import warnings

### Задание

1. Загрузите датасет. Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [3]:
targets=[]
messages=[]
with codecs.open('SMSSpamCollection.txt','r',"utf_8_sig") as data:
    for line in data.readlines():
        targets.append(line.split('\t')[0])
        messages.append(line.split('\t')[1].split('\n')[0])
        

In [6]:
targets=[ 1 if x=='spam' else 0 for x in targets]

2. Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [9]:
vm=CountVectorizer()
X=vm.fit_transform(messages)

In [10]:
X

<5574x8713 sparse matrix of type '<class 'numpy.int64'>'
	with 74169 stored elements in Compressed Sparse Row format>

3. Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [58]:
answer_index=1
fname='answer{}.txt'.format(answer_index)
def save_answerNum(fname,number):
    with open(fname,"w") as fout:
        fout.write(str(number))
    global answer_index
    answer_index+=1

In [59]:
warnings.filterwarnings('ignore')
cvs_mean=round(cross_val_score(LogisticRegression(random_state=2),X,targets,cv=10, scoring='f1').mean(),3)
print('Среднее качества классификации текста с помощью Логистической регресии: ', 
      cvs_mean)
save_answerNum(fname,cvs_mean)

Среднее качества классификации текста с помощью Логистической регресии:  0.925


4. А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [20]:
def clf_pipeline(vectorizer,classifier):
    return Pipeline([
        ('vectorizer',vectorizer),
        ('classifier',classifier)
    ])

In [21]:
clf=clf_pipeline(CountVectorizer(),LogisticRegression(random_state=2))

In [23]:
clf.fit(messages,targets)

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='warn', n_jobs=None,
             

In [61]:
prediction=clf.predict(["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",

"FreeMsg: Txt: claim your reward of 3 hours talk time",

"Have you visited the last lecture on physics?",

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",

"Only 99$"])
print(prediction)
save_answerNum(fname,prediction)

[1 1 0 0 0]


5.Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [68]:
ngrams=[(2,2),(3,3),(1,3)]
cvs_means=[]
for ngram in ngrams:
    cvs_mean=round(cross_val_score(clf_pipeline(CountVectorizer(ngram_range=ngram),
                        LogisticRegression(random_state=2)),
                          messages,targets,cv=10,scoring='f1').mean(),3)
    cvs_means.append(cvs_mean)
    print('Среднее качества классификации текста с помощью Логистической регресии при ngram= {}: '.format(ngram),
         cvs_mean)
save_answerNum(fname,cvs_means)


Среднее качества классификации текста с помощью Логистической регресии при ngram= (2, 2):  0.822
Среднее качества классификации текста с помощью Логистической регресии при ngram= (3, 3):  0.725
Среднее качества классификации текста с помощью Логистической регресии при ngram= (1, 3):  0.925


6. Повторите аналогичный п.5 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор.

In [70]:
cvs_means=[]
for ngram in ngrams:
    vm=CountVectorizer(ngram_range=ngram)
    X=vm.fit_transform(messages)
    cvs_mean=round(cross_val_score(MultinomialNB(),X,targets,cv=10,scoring='f1').mean(),3)
    cvs_means.append(cvs_mean)
    print('Среднее качества классификации текста с помощью Мультиномионального байесовского классификатора при ngram= {}: '.format(ngram),
          cvs_mean)
save_answerNum(fname,cvs_means)

Среднее качества классификации текста с помощью Мультиномионального байесовского классификатора при ngram= (2, 2):  0.645
Среднее качества классификации текста с помощью Мультиномионального байесовского классификатора при ngram= (3, 3):  0.379
Среднее качества классификации текста с помощью Мультиномионального байесовского классификатора при ngram= (1, 3):  0.888


7. Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [72]:
cvs_mean=round(cross_val_score(
        Pipeline([
                ("vectorizer", TfidfVectorizer(ngram_range=ngrams[2])),
                ("transformer", TfidfTransformer()),
                ("classifier",LogisticRegression(random_state=2))
            ]),
        messages,
        targets,
        cv=10,
        scoring='f1'
        ).mean(),3)
print('Среднее качества классификации текста с помощью Логистической регресси и TFidf при ngram= {}: '.format(ngrams[2]),
      cvs_mean)
answer=0
if abs(cvs_mean-cvs_means[2])<=0.01:
    answer=0
elif cvs_mean>cvs_means[2]:
    answer=1
else:
    answer=-1
save_answerNum(fname,answer)

Среднее качества классификации текста с помощью Логистической регресси и TFidf при ngram= (1, 3):  0.549
